In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import collections
import tensorflow as tf
import os
import random
from PIL import Image
import cv2
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [2]:
# Load the VGG model
vgg_model = VGG16(include_top=False)

In [3]:
# Freeze current VGG16 model
vgg_model.trainable = False

In [4]:
vgg_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

In [5]:
# Check datasets are found
print(os.listdir('..\\data_raw'))
cwd = os.getcwd() + "\\..\\data_raw\\"
print(cwd)

['Meta', 'Meta.csv', 'Test', 'Test.csv', 'Train', 'Train.csv']
C:\Users\phill\Dropbox\College stuff\Masters\Year2\COMP09012 - Machine Learning\Assignment 2\github\assignment2\..\data_raw\


In [6]:
# # Load Dataset
height = 48
width = 48
channels = 3
classes = 43
n_inputs = height*width*channels
data = []
labels = []

for i in range(classes):
    path = cwd + "\\Train\\" + str(i)
    print(path)
    Class = os.listdir(path)
    
    for a in Class:
        try:
            image = cv2.imread(path + "\\" + a)
            image_fromarray = Image.fromarray(image, 'RGB')
            resize_image = image_fromarray.resize((height, width))
            data.append(np.array(resize_image))
            labels.append(i)
        except:
            print("Error reading image.")

            
#Create numpy arrays from input data            
image_data = np.array(data)
image_labels = np.array(labels)

print(image_data.shape, image_labels.shape)


C:\Users\phill\Dropbox\College stuff\Masters\Year2\COMP09012 - Machine Learning\Assignment 2\github\assignment2\..\data_raw\\Train\0
C:\Users\phill\Dropbox\College stuff\Masters\Year2\COMP09012 - Machine Learning\Assignment 2\github\assignment2\..\data_raw\\Train\1
C:\Users\phill\Dropbox\College stuff\Masters\Year2\COMP09012 - Machine Learning\Assignment 2\github\assignment2\..\data_raw\\Train\2
C:\Users\phill\Dropbox\College stuff\Masters\Year2\COMP09012 - Machine Learning\Assignment 2\github\assignment2\..\data_raw\\Train\3
C:\Users\phill\Dropbox\College stuff\Masters\Year2\COMP09012 - Machine Learning\Assignment 2\github\assignment2\..\data_raw\\Train\4
C:\Users\phill\Dropbox\College stuff\Masters\Year2\COMP09012 - Machine Learning\Assignment 2\github\assignment2\..\data_raw\\Train\5
C:\Users\phill\Dropbox\College stuff\Masters\Year2\COMP09012 - Machine Learning\Assignment 2\github\assignment2\..\data_raw\\Train\6
C:\Users\phill\Dropbox\College stuff\Masters\Year2\COMP09012 - Machin

In [7]:
# # Split up the data in train and test
x_train, x_test, y_train, y_test = train_test_split(image_data, image_labels, test_size=0.2, random_state=42, shuffle=True)

In [8]:
# Normalalize data
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)
y_train = tf.keras.utils.to_categorical(y_train, 43)
y_test = tf.keras.utils.to_categorical(y_test, 43)
# x_train = x_train/255
# x_test = x_test/255
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(31367, 48, 48, 3)
(7842, 48, 48, 3)
(31367,)
(7842,)
(31367, 48, 48, 3)
(7842, 48, 48, 3)
(31367, 43)
(7842, 43)


In [9]:
# Add a classification Header
end = vgg_model.layers[-1].output
model = tf.keras.Model(vgg_model.input, end)

In [10]:
# model.trainable = True
# model.summary()

In [11]:
# Modelling
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(100, activation=tf.nn.relu))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(43, activation=tf.nn.softmax))


In [12]:
# Paramertise
model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [13]:
# print('Number of Total batches: %d' % tf.data.experimental.cardinality(train_dataset))

In [14]:
# # Setup Test and validation testsets
# total_batches = tf.data.experimental.cardinality(train_dataset)
# test_dataset = train_dataset.take(total_batches // 10)
# train_dataset = train_dataset.skip(total_batches // 10)
# validation_dataset = train_dataset.take(total_batches // 5)
# train_dataset = train_dataset.skip(total_batches // 5)

In [15]:
# print('Number of train batches: %d' % tf.data.experimental.cardinality(train_dataset))
# print('Number of test batches: %d' % tf.data.experimental.cardinality(test_dataset))
# print('Number of validation batches: %d' % tf.data.experimental.cardinality(validation_dataset))

In [16]:
# # Normalize
x_train = tf.keras.utils.normalize(x_train, axis=1)
x_test = tf.keras.utils.normalize(x_test, axis=1)
y_train = tf.keras.utils.normalize(y_train, axis=1)
y_test = tf.keras.utils.normalize(y_test, axis=1)

In [17]:
# # Configure Dataset for performance
# AUTOTUNE = tf.data.AUTOTUNE

# train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
# validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)
# test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)


In [18]:
# Train the model
model.fit(x_train, y_train, batch_size=256,
          validation_data=(x_test, y_test), epochs=10)

Epoch 1/10
123/123 [==============================] - 2s 14ms/step - loss: 3.3467 - accuracy: 0.1190 - val_loss: 2.0062 - val_accuracy: 0.4436
Epoch 2/10
123/123 [==============================] - 1s 9ms/step - loss: 2.0073 - accuracy: 0.4101 - val_loss: 1.2819 - val_accuracy: 0.6599
Epoch 3/10
123/123 [==============================] - 1s 9ms/step - loss: 1.4775 - accuracy: 0.5395 - val_loss: 0.9774 - val_accuracy: 0.7429
Epoch 4/10
123/123 [==============================] - 1s 9ms/step - loss: 1.2234 - accuracy: 0.6226 - val_loss: 0.8406 - val_accuracy: 0.7697
Epoch 5/10
123/123 [==============================] - 1s 9ms/step - loss: 1.0784 - accuracy: 0.6623 - val_loss: 0.7205 - val_accuracy: 0.8015
Epoch 6/10
123/123 [==============================] - 1s 9ms/step - loss: 0.9640 - accuracy: 0.6977 - val_loss: 0.6203 - val_accuracy: 0.8360
Epoch 7/10
123/123 [==============================] - 1s 10ms/step - loss: 0.9217 - accuracy: 0.7109 - val_loss: 0.5476 - val_accuracy: 0.8595
Epoc

In [19]:
# Evaluation
score = model.evaluate(x_test, y_test)
print(score)

246/246 [==============================] - 0s 851us/step - loss: 0.4641 - accuracy: 0.8826
[0.46407458186149597, 0.8825554847717285]


In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 6912)              0         
_________________________________________________________________
dense (Dense)                (None, 100)               691300    
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               12928     
_________________________________________________________________
dense_2 (Dense)              (None, 43)                5547      
Total params: 709,775
Trainable params: 709,775
Non-trainable params: 0
_________________________________________________________________


In [21]:
# class_names = train_dataset.class_names

# plt.figure(figsize=(10, 10))
# for images, labels in train_dataset.take(1):
#   for i in range(9):
#     ax = plt.subplot(3, 3, i + 1)
#     plt.imshow(images[i].numpy().astype("uint8"))
#     plt.title(class_names[labels[i]])
#     plt.axis("off")

In [22]:
# print('Number of validation batches: %d' % tf.data.experimental.cardinality(validation_dataset))
# print('Number of test batches: %d' % tf.data.experimental.cardinality(test_dataset))

In [23]:
#mnist = tf.keras.datasets.mnist
#(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [24]:
# model.summary()

In [25]:
# # Model Evaluation
# val_loss, val_acc = model.evaluate(x_test, y_test)
# print(val_loss, val_acc)

In [26]:
# # Plot and show
# plt.imshow(x_train[0])
# plt.show()